In [ ]:
# 1. Explore contours

from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
from pykrige.uk import UniversalKriging
import cmocean

plt.clf();                
cmap = cmocean.cm.balance;
fig, ax = plt.subplots(figsize=(8, 4))
patches = []
bottom = np.genfromtxt("bottom.txt", delimiter=" ");  
data = np.genfromtxt("2018.txt", delimiter=" ");  
data = data[::2, :]
contarr = [34.75,34.8,34.85]
gridx = np.arange(9, 65.0, 0.25)
gridy = np.arange(0.0, 450.0, 3)
UK = UniversalKriging(data[:, 1], data[:, 0], data[:, 2], variogram_model='linear',
                              drift_terms=['regional_linear'])
z, ss = UK.execute('grid', gridx, gridy)
plt.scatter(data[:, 1], data[:, 0],s=2)
rr, cc = np.meshgrid(gridx, gridy)
CS = ax.contour(rr, cc, z,100,levels=contarr,c=z, 
                 linewidths=0.5, cmap=cmocean.cm.balance,zorder = 2)
cl = ax.clabel(CS, CS.levels[::1],inline=False, fmt='%.2f',colors='black', fontsize=6)
points = plt.scatter(bottom[:,0],bottom[:,1],s=1)
plt.xlabel("Расстояние, км");
plt.ylabel("Глубина, м");
plt.xticks([w*10 for w in range(11)], [str(w*10) for w in range(11)]);
plt.xlim(0,105);
plt.ylim(0,450)
plt.grid (True , linestyle= '-', color='0.75',zorder=0)

ax.set_ylim(ax.get_ylim()[::-1])

plt.savefig("main_contours.png",bbox_inches='tight',pad_inches=0.05, dpi=300);
plt.close();


In [ ]:
# 2. Plot water mass polygon (above bottom) pb and all the considered contours from CS.allsegs[1]

from shapely.geometry import Polygon
pb = Polygon([(i[0], i[1]) for i in zip(bottom[:,0],bottom[:,1])])

num = len(CS.allsegs[1])
#number of contours
print(num)

fig, ax = plt.subplots(figsize=(8, 4))
for i in range(num):    
    dat0= CS.allsegs[1][i]
    plt.plot(dat0[:,0],dat0[:,1])    

plt.plot(*pb.exterior.xy)
ax.set_ylim(ax.get_ylim()[::-1])

print(len(CS.allsegs[1]))

In [ ]:

# 2 (continued). Plot water mass polygon (above bottom) pb and each the considered contours - CS.allsegs[1][0]
fig, ax = plt.subplots(figsize=(8, 4))

dat0= CS.allsegs[1][0]
li = [(i[0], i[1]) for i in zip(dat0[:,0],dat0[:,1])]
li.append((10,450))


p_p = Polygon(li)
plt.plot(*p_p.exterior.xy,"red")

pp_p = p_p.intersection(pb)
plt.plot(*pb.exterior.xy,"blue")
for p in pp_p:
    print(p.area*0.001)
    plt.plot(*p.exterior.xy,"brown");
    plt.fill(*p.exterior.xy,"yellow");

plt.plot(*p.exterior.xy,"brown");
plt.fill(*p.exterior.xy,"yellow");
ax.set_ylim(ax.get_ylim()[::-1])

In [ ]:
# 2 (continued). Plot water mass polygon (above bottom) pb and each the considered contours - CS.allsegs[1][1]

fig, ax = plt.subplots(figsize=(8, 4))

dat0= CS.allsegs[1][1]
li = [(i[0], i[1]) for i in zip(dat0[:,0],dat0[:,1])]

p_p = Polygon(li)
plt.plot(*p_p.exterior.xy,"red")

pp_p = p_p.intersection(pb)
plt.plot(*pb.exterior.xy,"blue")
p = pp_p
print(p.area*0.001)
plt.plot(*p.exterior.xy,"brown");
plt.fill(*p.exterior.xy,"yellow");

plt.plot(*p.exterior.xy,"brown");
plt.fill(*p.exterior.xy,"yellow");
ax.set_ylim(ax.get_ylim()[::-1])

In [ ]:
# 3. All together. 

from matplotlib import pyplot as plt
import numpy as np
from shapely.geometry import Polygon

from matplotlib.path import Path
from matplotlib.patches import PathPatch
import matplotlib
matplotlib.use('tkagg')

import scipy as sp
from matplotlib.patches import Polygon as plotpoly
from matplotlib.collections import PatchCollection
import cmocean

plt.clf();                
cmap = cmocean.cm.balance;
fig, ax = plt.subplots(figsize=(6, 4))
patches = []

total = 0
contarr = [34.75,34.8,34.85]

rr, cc = np.meshgrid(gridx, gridy) 
CSF = plt.contourf(rr, cc, z) # z from kriging before

#points = plt.scatter(bottom[:,0],bottom[:,1],s=1) #,c=data[:,2], s=10, cmap=cmap,zorder=2,vmin=valmin, vmax=valmax)
plt.xlabel("Расстояние, км");
plt.ylabel("Глубина, м");
plt.xticks([w*10 for w in range(11)], [str(w*10) for w in range(11)]);
plt.xlim(0,70);
plt.ylim(0,450)
plt.grid (linestyle= '-', color='0.75',zorder=0)

polygon = plotpoly(bottom, True)
patches.append(polygon)
p = PatchCollection(patches)               
p.zorder = 10                   
p.set(color = "lightgrey")
ax.add_collection(p)

dat0= CS.allsegs[1][0]  # 1st isoline
li = [(i[0], i[1]) for i in zip(dat0[:,0],dat0[:,1])]
li.append((10,450))
p_p = Polygon(li)
pp_p = p_p.intersection(pb)
for p in pp_p:  # two shapes for the 1st isoline
    total += p.area*0.001
    plt.fill(*p.exterior.xy,"yellow");
plt.fill(*p.exterior.xy,"yellow");

dat0= CS.allsegs[1][1] # 2nd isoline
li = [(i[0], i[1]) for i in zip(dat0[:,0],dat0[:,1])]
p_p = Polygon(li) # one shape for the 2nd isoline
pp_p = p_p.intersection(pb)
p = pp_p
total += p.area*0.001
plt.fill(*p.exterior.xy,"yellow");

plt.scatter(data[:, 1], data[:, 0],s=1,c="black")
CS2 = ax.contour(rr, cc, z,10,levels=[contarr[1]],colors="red", 
                 linewidths=0.5, zorder = 2)
cl = ax.clabel(CS2, CS2.levels[::1],inline=False, fmt='%.1f',colors='brown', fontsize=14,manual = True)

ax.text(20, 350, 'Площадь ≈ '+str(float("{0:.1f}".format(total))) +' км²' , style='italic', color='green', fontsize=15,zorder = 20)
ax.text(55, 350, '2018', style='italic', color='red', fontsize=18,zorder = 20)
ax.set_ylim(ax.get_ylim()[::-1])
plt.savefig("sval_2018.jpg",bbox_inches='tight',pad_inches=0.1, dpi=600);
plt.close();